In [ ]:
import pandas as pd

def carregar_dados_ideb(caminho):
    """Carrega dados do IDEB"""
    return pd.read_csv(caminho, encoding='ISO-8859-1')

In [ ]:
# Visualizar estrutura
print(df.head())
print("\nColunas disponíveis:", df.columns.tolist())

NameError: name 'df' is not defined

In [ ]:
df = pd.read_csv("/content/br_inep_ideb_brasil.csv")

In [ ]:
df.head()

,ano,rede,ensino,anos_escolares,taxa_aprovacao,indicador_rendimento,nota_saeb_matematica,nota_saeb_lingua_portuguesa,nota_saeb_media_padronizada,ideb,projecao
0,2009,total,medio,todos (1-4),75.9,0.795302,274.72,268.82,4.572342,3.6,3.5
1,2013,total,medio,todos (1-4),80.1,0.823323,270.14,264.05,4.436750,3.7,3.9
2,2021,total,medio,todos (1-4),90.8,0.900923,271.00,275.97,4.626981,4.2,NaN
3,2007,total,medio,todos (1-4),74.1,0.778403,272.89,261.39,4.435226,3.5,3.4
4,2017,total,medio,todos (1-4),83.1,0.838888,270.63,268.51,4.510258,3.8,4.7


In [ ]:
print("\nColunas disponíveis:", df.columns.tolist())


Colunas disponíveis: ['ano', 'rede', 'ensino', 'anos_escolares', 'taxa_aprovacao', 'indicador_rendimento', 'nota_saeb_matematica', 'nota_saeb_lingua_portuguesa', 'nota_saeb_media_padronizada', 'ideb', 'projecao']


#Pré-processamento

In [ ]:
from sklearn.preprocessing import StandardScaler

def preprocessar_dados(df):
    """Filtra e normaliza os dados relevantes"""

    # Filtro: último ano disponível e ensino médio apenas
    df = df[(df['ano'] == df['ano'].max()) & (df['ensino'] == 'medio')].copy()

    # Mapeia tipos de rede
    redes_dict = {'municipal': 0, 'estadual': 1, 'privada': 2, 'total': 3}
    df['rede_num'] = df['rede'].map(redes_dict)

    # Remove linhas com dados faltantes
    features = ['rede_num', 'taxa_aprovacao',
                'nota_saeb_matematica', 'nota_saeb_lingua_portuguesa']
    df = df.dropna(subset=features + ['ideb'])

    # Normalização
    scaler = StandardScaler()
    df_norm = pd.DataFrame(
        scaler.fit_transform(df[features]),
        columns=[f"{col}_norm" for col in features]
    )

    return pd.concat([df.reset_index(drop=True), df_norm], axis=1)

#Modelagem

In [ ]:
from sklearn.neighbors import NearestNeighbors

def treinar_modelo(df):
    """Treina o modelo de vizinhos mais próximos"""
    features_norm = ['rede_num_norm', 'taxa_aprovacao_norm',
                     'nota_saeb_matematica_norm', 'nota_saeb_lingua_portuguesa_norm']

    X = df[features_norm].values
    model = NearestNeighbors(n_neighbors=3, metric='cosine')
    model.fit(X)
    return model

#Recomendação

In [ ]:
import numpy as np

def recomendar_similares(model, df, rede_nome, n=3):
    """Recomenda escolas similares com base na rede"""
    redes_dict = {'municipal': 0, 'estadual': 1, 'privada': 2, 'total': 3}
    rede_num = redes_dict.get(rede_nome)

    if rede_num is None:
        print("Rede inválida.")
        return pd.DataFrame()

    target_df = df[df['rede_num'] == rede_num].copy()
    if target_df.empty:
        print("Nenhuma escola encontrada para essa rede.")
        return pd.DataFrame()

    features_norm = ['rede_num_norm', 'taxa_aprovacao_norm',
                     'nota_saeb_matematica_norm', 'nota_saeb_lingua_portuguesa_norm']

    distances, indices = model.kneighbors(
        target_df[features_norm],
        n_neighbors=min(n, len(target_df))
    )

    recomendacoes = target_df.iloc[indices[0]][['rede', 'ideb', 'taxa_aprovacao',
                                                 'nota_saeb_matematica', 'nota_saeb_lingua_portuguesa']]
    recomendacoes['similaridade'] = 1 / (1 + distances[0])

    return recomendacoes.sort_values('similaridade', ascending=False)

In [ ]:
def recomendar_similares(df, modelo, rede_escolhida, top_n=5):
    filtro = df[df['rede'] == rede_escolhida]

    if filtro.empty:
        print(f"Nenhum dado encontrado para a rede: {rede_escolhida}")
        return

    idx = filtro.index[0]
    similares = modelo[idx]

    indices_recomendados = similares.argsort()[::-1][1:top_n+1]  # ignora o próprio
    recomendados = df.iloc[indices_recomendados]

    print(recomendados[['ano', 'rede', 'ensino', 'ideb']])

In [ ]:
print(df['rede'].unique())

['total' 'privada' 'publica' 'estadual' 'municipal']


#Execução principal

In [ ]:
from sklearn.neighbors import NearestNeighbors

def treinar_modelo(df):
    colunas_numericas = df.select_dtypes(include=[np.number])
    modelo = NearestNeighbors(metric='cosine')
    modelo.fit(colunas_numericas)
    return modelo

In [ ]:
def recomendar_similares(modelo, df, rede_escolhida, top_n=5):
    filtro = df[df['rede'] == rede_escolhida]

    if filtro.empty:
        print(f"Nenhum dado encontrado para a rede: {rede_escolhida}")
        return pd.DataFrame()

    idx = filtro.index[0]  # pega a primeira ocorrência
    try:
        distancias, indices = modelo.kneighbors([df.iloc[idx]], n_neighbors=top_n + 1)
        recomendados = df.iloc[indices[0][1:]]  # ignora o próprio ponto (índice 0)
        return recomendados[['ano', 'rede', 'ensino', 'ideb']]
    except Exception as e:
        print(f"Erro ao gerar recomendação: {e}")
        return pd.DataFrame()

In [ ]:
def preprocessar_dados(df):
    df = df.copy()

    # Converte colunas para o formato correto, se necessário
    df['ano'] = df['ano'].astype(int)

    # Filtra apenas o ensino médio
    df = df[df['ensino'] == 'medio']

    # Reset index para segurança
    df = df.reset_index(drop=True)

    # Trata valores ausentes nas colunas numéricas
    colunas_numericas = df.select_dtypes(include=[np.number]).columns
    df[colunas_numericas] = df[colunas_numericas].fillna(df[colunas_numericas].mean())

    return df

In [ ]:
print("Colunas com NaN antes de tratar:")
print(df.isna().sum())

Colunas com NaN antes de tratar:
ano                             0
rede                            0
ensino                          0
anos_escolares                  0
taxa_aprovacao                  0
indicador_rendimento            0
nota_saeb_matematica            0
nota_saeb_lingua_portuguesa     0
nota_saeb_media_padronizada     0
ideb                            0
projecao                       28
dtype: int64


In [ ]:
def treinar_modelo(df):
    colunas_numericas = df.select_dtypes(include=[np.number]).drop(columns=['projecao'])  # <- exclui a coluna
    modelo = NearestNeighbors(metric='cosine')
    modelo.fit(colunas_numericas)
    return modelo

In [ ]:
def main():
    caminho = '/content/br_inep_ideb_brasil.csv'

    print("🔹 Carregando dados...")
    df = carregar_dados_ideb(caminho)
    print(f"Dados carregados! Dimensão: {df.shape}")

    print("\n🔹 Pré-processando...")
    try:
        df_proc = preprocessar_dados(df)
        print("Pré-processamento concluído.")
    except Exception as e:
        print(f"Erro no pré-processamento: {e}")
        return

    print("\n🔹 Treinando modelo...")
    try:
        modelo = treinar_modelo(df_proc)
        print("Modelo treinado com sucesso.")
    except Exception as e:
        print(f"Erro no treinamento: {e}")
        return

    print("\n🔹 Gerando recomendações...\n")
    redes_para_testar = ['estadual', 'municipal', 'privada']

    for rede in redes_para_testar:
        print(f"▶ Recomendação para rede: {rede}")
        resultado = recomendar_similares(modelo, df_proc, rede)
        if not resultado.empty:
            print(resultado.to_markdown(tablefmt='grid'))
        else:
            print("Nenhuma recomendação encontrada.\n")

if __name__ == '__main__':
    main()

🔹 Carregando dados...
Dados carregados! Dimensão: (126, 11)

🔹 Pré-processando...
Pré-processamento concluído.

🔹 Treinando modelo...
Modelo treinado com sucesso.

🔹 Gerando recomendações...

▶ Recomendação para rede: estadual
+----+-------+----------+----------+--------+
|    |   ano | rede     | ensino   |   ideb |
+====+=======+==========+==========+========+
| 27 |  2007 | estadual | medio    |    3.2 |
+----+-------+----------+----------+--------+
| 25 |  2007 | publica  | medio    |    3.2 |
+----+-------+----------+----------+--------+
| 18 |  2005 | publica  | medio    |    3.1 |
+----+-------+----------+----------+--------+
| 28 |  2005 | estadual | medio    |    3   |
+----+-------+----------+----------+--------+
| 31 |  2013 | estadual | medio    |    3.4 |
+----+-------+----------+----------+--------+
▶ Recomendação para rede: municipal
Nenhum dado encontrado para a rede: municipal
Nenhuma recomendação encontrada.

▶ Recomendação para rede: privada
+----+-------+---------+-

In [ ]:
def recomendar_similares(modelo, df, rede_escolhida, top_n=5):
    filtro = df[df['rede'] == rede_escolhida]

    if filtro.empty:
        print(f"Nenhum dado encontrado para a rede: {rede_escolhida}")
        return pd.DataFrame()

    # Seleciona apenas as colunas numéricas que foram usadas no treinamento
    colunas_numericas = df.select_dtypes(include=[np.number]).drop(columns=['projecao'])  # ou mantenha se tiver imputado
    dados_numericos = colunas_numericas.loc[filtro.index]

    # Aplica o modelo
    distancias, indices = modelo.kneighbors(dados_numericos, n_neighbors=top_n)

    resultados = df.iloc[indices[0]]
    return resultados[['ano', 'rede', 'ensino', 'ideb']]  # ou personalize aqui